In [6]:
import pandas as pd
import os
import sys
import json

In [7]:
if sys.platform.startswith("linux"):
    data_dir = "/bask/projects/v/vjgo8416-amber/data/gbif_download_standalone/"
elif sys.platform == "darwin":
    data_dir = "/Users/lbokeria/Documents/projects/gbif-species-trainer-data/"
else:
    print("Not linux or mac!")

checklist_name = "uksi-moths-keys-nodup"

df = pd.read_csv(os.path.join("../species_checklists",checklist_name+".csv"))

In [8]:
df["n_imgs"] = ""
for idx, row in df.iterrows():
    
    family  = row["family_name"]
    genus   = row["genus_name"]
    species = row["gbif_species_name"]
    
    # Check if directory exists 
    species_dir = (
        os.path.join(data_dir,"gbif_images","sandbox",family,genus,species)
        )
    
    if os.path.isdir(species_dir):
        n_images_on_disk = len(
            [f for f in os.listdir(species_dir) if f.lower().endswith('.jpg') ]
            )
        # print(f"{species} Count files method has", n_images_on_disk, "images")
        
        # Load metadata
        try:
            
            with open(os.path.join(species_dir,"meta_data.json")) as file:
            
                meta_data = json.load(file)
            
            # Count the number of images for this species
            n_imgs_metadata = 0

            for key, value in meta_data.items():
                if value.get("image_is_downloaded") == True:
                    n_imgs_metadata += 1
            # print(f"{species} Count json metadata has", n_imgs_metadata, "images") 
            
            try:
                # 2nd way of counting images
                md2 = pd.read_json(
                    os.path.join(species_dir,"meta_data.json"), orient = 'index'
                    )
                
                if md2.empty:
                    md2_n_imgs_downloaded = 0
                else:
                    md2_n_imgs_downloaded = md2["image_is_downloaded"].sum()

                # print(f"{species} Count dataframe metadata has", md2_n_imgs_downloaded, "images")            

                if md2_n_imgs_downloaded != n_imgs_metadata:
                    print(f"Two ways of calculating metadata images don't match for {species_dir}")
            except Exception as e:
                print(e)
                print(f"{species} error counting dataframe way: {e}")
                        
            # Do n images match?
            if n_images_on_disk == n_imgs_metadata:
                # print(f"N images match for {species_dir}")
                pass
            else:
                print(
                    f"N images MISmatch. File count is {n_images_on_disk}, "
                    f"metadata has {n_imgs_metadata}, species: {species}"
                )
        except Exception as e:
            pass
            print(f"No metadata for {species_dir}")
            
    else:
        
        n_images_on_disk = 0
    
    # print(f"{species} has {n_images_on_disk} images")
    
    # Record this
    df.loc[idx, "n_imgs"] = n_images_on_disk
    
    

N images MISmatch. File count is 1652, metadata has 1651, species: Catoptria falsella
N images MISmatch. File count is 1309, metadata has 922, species: Watsonalla cultraria
N images MISmatch. File count is 1586, metadata has 1585, species: Catocala fraxini
N images MISmatch. File count is 1857, metadata has 1856, species: Hypena proboscidalis
N images MISmatch. File count is 1913, metadata has 1912, species: Lymantria monacha
N images MISmatch. File count is 1896, metadata has 1895, species: Scoliopteryx libatrix
N images MISmatch. File count is 1001, metadata has 1000, species: Spilosoma lubricipeda
N images MISmatch. File count is 1583, metadata has 1582, species: Agriopis marginaria
N images MISmatch. File count is 1557, metadata has 1556, species: Angerona prunaria
N images MISmatch. File count is 1456, metadata has 1455, species: Archiearis parthenias
N images MISmatch. File count is 1685, metadata has 1000, species: Biston strataria
N images MISmatch. File count is 1312, metadata

In [15]:
species_dir = "/bask/homes/r/rybf4168/vjgo8416-amber/data/gbif_download_standalone/gbif_images/tests/Adelidae/Nemophora/Nemophora aurisparsella"

with open(os.path.join(species_dir,"meta_data.json")) as file:

    meta_data = json.load(file)

meta_data

{}

In [20]:
a = pd.read_json(os.path.join(species_dir,"meta_data.json"), orient='index')
a.empty

True

In [ ]:
# Save the df
df.to_csv(os.path.join("../data_stats_files/","data_stats_"+checklist_name+".csv"))